In [1]:
import networkx as nx
import pandas as pd
import yaml

from collections import defaultdict
from bs4 import BeautifulSoup


In [2]:
def track_term(term, concept, lower=True):
    if pd.isna(term) or term == '\\N' or term is None:
        return
    
    term = str(term)
    term2concept[term] = concept
    
    if lower:
        term2concept[term.lower()] = concept
    
def track_concept(name, category):
    if pd.isna(name) or name == '\\N' or name is None:
        return
    
    name = str(name)
    concept = '_'.join(name.lower().split())
    track_term(name, concept)
    concept2name[concept] = name
    concept2category[concept] = category
    return concept

def find_concept(name):
    if pd.isna(name):
        return
    
    name = str(name)
    concept = '_'.join(name.lower().split())
    if concept in concept2name:
        return concept
    return

def link_concepts(concept1, concept2):
    concept2neighbour[concept1].add(concept2)
    concept2neighbour[concept2].add(concept1)

# Categories

In [3]:
categories = {
    'Airport':'Airport [A]',
    'Airline':'Airline [B]',
    'Aircraft':'Aircraft [C]',
    'Components': 'Components [D]',
    'Engine':'Engine [E]',
    'Fault': 'Fault [F]',
    'Geographic Location':'Geographic Location [G]',
    'Manufacturer':'Manufacturer [H]',
    'Material': 'Material [I]',
    'Person':'Person [J]',
    'Unit': 'Unit [L]',
}

term2concept = {}
concept2name = {}
concept2category = {}
concept2neighbour = defaultdict(lambda: set())

In [4]:
for term, category in categories.items():
    concept = track_concept(term, category)
    link_concepts(concept, 'root')

# Airport Dataset
* Georgraphic Locations: Continents, Countries, Cities
* Airports: Name, IATA, ICAO Codes

In [5]:
url = 'https://raw.githubusercontent.com/jpatokal/openflights/master/data/airports.dat'
columns = ['airport', 'city', 'country', 'iata', 'icao', 'latitude', 'longitude', 'altitude', 'utf_offset', 'dailylight_savings_time', 'timezone', 'type', 'source']
airport_df = pd.read_csv(url, names=columns)
airport_df.head()

,airport,city,country,iata,icao,latitude,longitude,altitude,utf_offset,dailylight_savings_time,timezone,type,source
1,Goroka Airport,Goroka,Papua New Guinea,GKA,AYGA,-6.081690,145.391998,5282,10,U,Pacific/Port_Moresby,airport,OurAirports
2,Madang Airport,Madang,Papua New Guinea,MAG,AYMD,-5.207080,145.789001,20,10,U,Pacific/Port_Moresby,airport,OurAirports
3,Mount Hagen Kagamuga Airport,Mount Hagen,Papua New Guinea,HGU,AYMH,-5.826790,144.296005,5388,10,U,Pacific/Port_Moresby,airport,OurAirports
4,Nadzab Airport,Nadzab,Papua New Guinea,LAE,AYNZ,-6.569803,146.725977,239,10,U,Pacific/Port_Moresby,airport,OurAirports
5,Port Moresby Jacksons International Airport,Port Moresby,Papua New Guinea,POM,AYPY,-9.443380,147.220001,146,10,U,Pacific/Port_Moresby,airport,OurAirports


In [6]:
for i, row in airport_df.iterrows():
    continent = row.timezone.split('/')[0]
    continent_concept = track_concept(continent, categories['Geographic Location'])
    country_concept = track_concept(row.country, categories['Geographic Location'])
    city_concept = track_concept(row.city, categories['Geographic Location'])
    
    airport_concept = track_concept(row.airport, categories['Airport'])
    track_term(row.iata, airport_concept, lower=False)
    track_term(row.icao, airport_concept, lower=False)
    
    link_concepts(continent_concept, 'geographic_location')
    link_concepts(continent_concept, country_concept)
    link_concepts(country_concept, city_concept)
    link_concepts(airport_concept, city_concept)
    link_concepts(airport_concept, 'airport')

In [7]:
print('Num countries', airport_df.country.nunique())
print('Num cities', airport_df.city.nunique())
print('Num airports', airport_df.airport.nunique())

Num countries 237
Num cities 6955
Num airports 7658


# Airline Dataset

In [8]:
url = 'https://raw.githubusercontent.com/jpatokal/openflights/master/data/airlines.dat'
columns = ['name', 'alias', 'iata', 'icao', 'callsign', 'country', 'active']
airline_df = pd.read_csv(url, names=columns)
airline_df = airline_df[airline_df.active == 'Y']
airline_df = airline_df[airline_df.name != 'Private flight']
airline_df = airline_df[airline_df.name != 'Unknown']
airline_df = airline_df[airline_df.name != 'Hello']
airline_df.head()

,name,alias,iata,icao,callsign,country,active
3,1Time Airline,\N,1T,RNX,NEXTIME,South Africa,Y
10,40-Mile Air,\N,Q5,MLA,MILE-AIR,United States,Y
13,Ansett Australia,\N,AN,AAA,ANSETT,Australia,Y
14,Abacus International,\N,1B,NaN,NaN,Singapore,Y
21,Aigle Azur,\N,ZI,AAF,AIGLE AZUR,France,Y


In [9]:
len(airline_df)

1252

In [10]:
mapper = {
    'Republic of Korea': 'South Korea',
    'Democratic People\'s Republic of Korea': 'North Korea',
    'Canadian Territories': 'Canada',
    'Russian Federation': 'Russia',
    'Macao': 'Macau',
    'Republic of the Congo': 'Congo (Brazzaville)',
    'Hong Kong SAR of China': 'Hong Kong',
    'Ivory Coast': 'Cote d\'Ivoire',
    'Lao Peoples Democratic Republic': 'Laos',
    'Syrian Arab Republic': 'Syria',
}

airline_df.country = airline_df.country.map(lambda x: mapper[x] if x in mapper else x)

In [11]:
for i, row in airline_df.iterrows():
    airline_concept = track_concept(row['name'], categories['Airline'])
    track_term(row.iata, airline_concept, lower=False)
    track_term(row.icao, airline_concept, lower=False)
    track_term(row.callsign, airline_concept)
    link_concepts(airline_concept, 'airline')
    
    country_concept = find_concept(row.country)
    if country_concept:
        link_concepts(airline_concept, country_concept)
    else:
        print('Dropped', country_concept, row.country)

Dropped None  S.A.
Dropped None ALASKA
Dropped None AVIANCA
Dropped None nan
Dropped None DRAGON
Dropped None \N


# Aircraft Dataset

In [12]:
url = 'https://raw.githubusercontent.com/jpatokal/openflights/master/data/planes.dat'
columns = ['name', 'iso', 'dafif']
aircraft_df = pd.read_csv(url, names=columns)
aircraft_df['manufacturer'] = aircraft_df.name.map(lambda x: x.split()[0])
aircraft_df.head()

,name,iso,dafif,manufacturer
0,Aerospatiale (Nord) 262,ND2,N262,Aerospatiale
1,Aerospatiale (Sud Aviation) Se.210 Caravelle,CRV,S210,Aerospatiale
2,Aerospatiale SN.601 Corvette,NDC,S601,Aerospatiale
3,Aerospatiale/Alenia ATR 42-300,AT4,AT43,Aerospatiale/Alenia
4,Aerospatiale/Alenia ATR 42-500,AT5,AT45,Aerospatiale/Alenia


In [13]:
for i, row in aircraft_df.iterrows():
    aircraft_concept = track_concept(row['name'], categories['Aircraft'])
    track_term(row.iso, aircraft_concept, lower=False)
    track_term(row.dafif, aircraft_concept, lower=False)
    
    manufacturer = row.manufacturer.split('/')[0]
    manufacturer_concept = track_concept(manufacturer, categories['Manufacturer'])
    
    link_concepts('aircraft', aircraft_concept)
    link_concepts('manufacturer', manufacturer_concept)
    link_concepts(aircraft_concept, manufacturer_concept)

# Turbine Dataset

In [14]:
url = 'https://www.easa.europa.eu/sites/default/files/dfu/edb-emissions-databank%20v26B-NewFormat%20%28web%29.xlsx'
turbine_df = pd.read_excel(url, sheet_name='ICAO databank (new format)')
turbine_df.columns = turbine_df.columns.map(lambda x: '_'.join(x.lower().split()))
turbine_df.head()

,uid_no,dtb_no,manufacturer,engine_identification,combustor_description,eng_type,b/p_ratio,pressure_ratio,rated_thrust_(kn),data_status,...,final_test_date,compliance_with_fuel_venting_requirements,remark_1,remark_2,remark_3,remark_4,remark_5,remark_6,remark_7,remark_8
0,1AS001,1,Allied Signal,TFE731-2-2B,NaN,TF,2.64,13.90,15.60,PR,...,1976-04-01,NaN,Not required to meet GASEOUS regulations,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1AS002,1,Allied Signal,TFE731-3,NaN,TF,2.64,14.30,16.50,PR,...,1976-08-01,NaN,Not required to meet GASEOUS regulations,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,4AL003,4,Rolls-Royce Corporation,AE3007A,NaN,MTF,5.23,18.08,33.73,C,...,1995-04-19,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,6AL005,6,Rolls-Royce Corporation,AE3007A1,Type 1,MTF,4.77,17.97,34.91,R,...,1995-04-19,NaN,For effectivity see Rolls-Royce Notice to Oper...,Results are based on Rolls-Royce report EDR 15...,AE3007A1 data in this report replace data repo...,NaN,NaN,NaN,NaN,NaN
4,6AL006,6,Rolls-Royce Corporation,AE3007A1,Type 2,MTF,4.77,17.97,34.91,C,...,2002-08-29,NaN,For effectivity see Rolls-Royce Notice to Oper...,Data in this form for AE3007A1 egines fitted w...,Results based on Rolls-Royce report EDR 19972,NaN,NaN,NaN,NaN,NaN


In [15]:
## RR Test Facilities:
mapper = {
    'Indianapolis, Indiana': 'Indianapolis',
    'British Airways Test Facility, Heathrow': 'London',
    'SINFIN-Derby': 'Derby'
}

In [16]:
rr_concept = track_concept('Rolls Royce', categories['Manufacturer'])
track_term('Rolls Royce', rr_concept)
track_term('Rolls-Royce', rr_concept)
track_term('RR', rr_concept)
track_term('RRD', rr_concept)

for i, row in turbine_df.iterrows():
    engine_concept = track_concept(row.engine_identification, categories['Engine'])
    
    manufacturer = row.manufacturer.replace('-', ' ')
    manufacturer_concept = track_concept(manufacturer, categories['Manufacturer'])
    
    link_concepts(engine_concept, 'engine')
    link_concepts(manufacturer_concept, 'manufacturer')
    link_concepts(engine_concept, manufacturer_concept)
    
    if 'rolls' in manufacturer_concept:
        facility = mapper[row.test_location] if row.test_location in mapper else row.test_location
        facility_concept = track_concept(facility, categories['Geographic Location'])

        link_concepts(rr_concept, manufacturer_concept)
        link_concepts(facility_concept, manufacturer_concept)
        link_concepts(facility_concept, engine_concept)

# Registered Aircrafts

In [17]:
url = 'https://opensky-network.org/datasets/metadata/aircraftDatabase.csv'
registration_df = pd.read_csv(url)
registration_df = registration_df[registration_df.registration.notna()]
registration_df = registration_df[registration_df.manufacturericao.notna()]
registration_df.head()

/home/phhage/anaconda3/envs/torchenv/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3051: DtypeWarning: Columns (14) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,icao24,registration,manufacturericao,manufacturername,model,typecode,serialnumber,linenumber,icaoaircrafttype,operator,...,status,built,firstflightdate,seatconfiguration,engines,modes,adsb,acars,notes,categoryDescription
2,ae267b,6533,VOUGHT,Aerospatiale,MH-65C Dolphin,AS65,6182,NaN,H2T,NaN,...,NaN,NaN,NaN,NaN,NaN,False,False,False,NaN,No ADS-B Emitter Category Information
5,391927,F-GGJH,ROBIN,Robin,DR.400 160 Chevalier,DR40,1795,NaN,L1P,NaN,...,NaN,NaN,NaN,NaN,NaN,False,False,False,NaN,NaN
6,a61cf6,N493TR,CIRRUS,Cirrus Design Corp,SR22T,S22T,0776,NaN,L1P,NaN,...,NaN,2014-01-01,NaN,NaN,CONT MOTOR TSIO-550-K,False,False,False,NaN,Light (< 15500 lbs)
8,aa6735,N77FK,GULFSTREAM AEROSPACE,Gulfstream Aerospace,GIV SP,GLF4,1357,NaN,L2J,NaN,...,NaN,NaN,NaN,NaN,NaN,False,False,False,NaN,Small (15500 to 75000 lbs)
9,3d3191,D-ERAF,GROB,Grob,G-115 E,G115,82085/E,NaN,L1P,NaN,...,NaN,NaN,NaN,NaN,NaN,False,False,False,NaN,NaN


In [18]:
def filter_aircrafts(row):
    return row.registration.startswith('EI') or\
        row.registration.startswith('VH') or\
        row.registration.startswith('VX') or\
        row.registration.startswith('ex') or\
        'rolls' in str(row.engines)

registration_df = registration_df[registration_df.apply(filter_aircrafts, axis=1)]

In [19]:
import re

def strip(text):
    return text.replace(u'\xa0', u' ').strip()

def strip_multiply(text):
    for i in range(5):
        text = text.replace(f'{i}x', '')
        text = text.replace(f'{i} x', '')
    
    return text

def strip_series(text):
    text = text.split('SERIE')[0]
    text = text.split('/')[0]
    
    return text

def strip_paranthesis(text):
    return re.sub(r'\(.*\)','', text)

def filter_short(text):
    return len(text) > 3

def filter_alpha(text):
    return any(c.isdigit() for c in text)

def split_manufacturer(text):
    stop = set(['trent', 'ae', 'turbomeca'])
    tokens = text.split()
    i = 0
    
    for i, t in enumerate(tokens):
        if any(c.isdigit() for c in t) or t.lower() in stop:
            break
    
    manufacturer = ' '.join(map(lambda x: x.capitalize(), tokens[:i]))
    manufacturer = rr_manufacturer(manufacturer)
    engine = ' '.join(tokens[i:])
    engine = rr_engine(engine)
    return (manufacturer, engine)

def rr_manufacturer(text):
    if 'rolls' in text.lower() or 'rr' in text.lower():
        return 'Rolls Royce'
    elif len(text) > 2:
        return text
    else:
        return ''
    
def rr_engine(text):
    text = text.replace('.', '-')
    text = text.replace('- ', '-')
    text = text.replace('.', '')
    text = text.split('ROLLS-ROYCE')[0]
    text = text.split('Trent')[0]
    text = text.split('TRENT')[0]
    text = text.strip()
    return text


def clean(text):
    text = BeautifulSoup(str(text)).get_text()
    text = strip_paranthesis(text)
    text = strip_multiply(text)
    text = strip_series(text)
    text = strip(text)
    
    if filter_short(text) and filter_alpha(text):
        return split_manufacturer(text)
    return ('', '')

registration_df['turbine_manufacturer'], registration_df['turbine'] = zip(*registration_df.engines.map(clean))

In [20]:
for i, row in registration_df.iterrows():
    aircraft_manufacturer_concept = track_concept(row.manufacturername, categories['Manufacturer'])
    aircraft_concept = track_concept(row.model, categories['Aircraft'])
    track_term(row.icaoaircrafttype, aircraft_concept)
    registered_aircraft_concept = track_concept(row.registration, categories['Aircraft'])
    
    link_concepts(aircraft_concept, aircraft_manufacturer_concept)
    link_concepts(aircraft_concept, registered_aircraft_concept)
    link_concepts(aircraft_manufacturer_concept, 'manufacturer')
    link_concepts(aircraft_concept, 'aircraft')
    
    if len(row.turbine) > 2:
        engine_concept = track_concept(row.turbine, categories['Engine'])
        engine_manufacturer_concept = track_concept(row.turbine_manufacturer, categories['Manufacturer'])

        link_concepts(engine_concept, 'engine')
        link_concepts(aircraft_manufacturer_concept, 'manufacturer')
        link_concepts(engine_concept, engine_manufacturer_concept)
        link_concepts(engine_concept, registered_aircraft_concept)

## Rolls Royce Engine Data
Add additional engines from case backlog

In [21]:
cases_df = pd.read_pickle('../data/graph/aviation/cases.pkl')

In [22]:
engines = ['BR710A2-20', 'BR710C4-11', 'BR715', 'BR725A1-12']
airlines = cases_df.operator.unique()

for engine in engines:
    concept = track_concept(engine, categories['Engine'])
    link_concepts('rolls_royce', concept)
    
for airline in airlines:
    concept = track_concept(airline, categories['Airline'])
    link_concepts('airline', concept)

In [23]:
cases_df

,id,owner_id,rolls_royce_aircraft_id,state,priority,severity,summary,description,next_flight_at,return_to_service_at,...,left_affected_engine_is_lease_engine,right_affected_engine_is_lease_engine,left_engine_serial_number,right_engine_serial_number,aircraft_serial_number,location_rolls_royce_id,operational_effect_descriptions,operational_symptom_descriptions,operational_reaction_descriptions,operator
0,21629,17,2394,Completed,2,3.0,FADEC BUS FAIL L A & B,Repeat write up. We received several CAS messa...,2016-11-14 20:00:43+00:00,2016-11-14 19:10:43+00:00,...,0.0,0.0,CAE330443,CAE330395,750-0195,3092,NaN,NaN,NaN,NetJets Aviation Inc.
1,21632,30,5616,Completed,1,3.0,Right FMU Fuel Leak from Heat Shield,Right FMU leaking from heat shield.,2016-11-15 17:00:46+00:00,2016-11-15 17:00:46+00:00,...,0.0,0.0,25485,25484,6188,2912,NaN,NaN,NaN,The Prudential Insurance Company
2,21638,17,2402,Completed,1,3.0,RH Eng ITT 900 DegC With Engine Static - Prior...,With engines shutdown prior to first start of ...,2016-11-15 01:00:21+00:00,2016-11-14 23:00:21+00:00,...,0.0,0.0,CAE330442,CAE330437,750-0203,2968,NaN,NaN,NaN,NetJets Aviation Inc.
3,21641,70,5568,Completed,2,NaN,"RRD-P/N8H0134G ""SWITCH,LP FUEL"" qty. 1 each re...",No calls. Customer made inquiry to OSD Spares....,2016-11-19 16:59:29+00:00,2016-11-18 17:00:29+00:00,...,0.0,0.0,25465,25464,6178,3353,NaN,NaN,NaN,"SINO JET (BEIJING) CO., LTD"
4,21644,111,2642,Completed,2,NaN,"Spares request on various p/n , see ""full issu...",AOG Spares request for :\r\n- DFYF000068VCT10...,2016-11-17 11:00:51+00:00,2016-11-17 11:00:52+00:00,...,0.0,0.0,12331,12330,9110,239,NaN,NaN,NaN,Jet Story Sp. z o.o.
5,21647,56,5835,Completed,3,NaN,Review NTO 181 - Correct Parking and Mooring,"Review of NTO 181, to include moisture damage ...",2016-11-30 08:00:38+00:00,2016-11-30 08:00:38+00:00,...,0.0,0.0,NaN,NaN,BR710_Fleet_GAC,5657,NaN,NaN,NaN,NaN
6,21650,106,4677,Completed,1,3.0,TRU corrosion,We found during TRs inspection corrosion on th...,2016-11-16 12:39:46+00:00,2016-11-15 12:39:46+00:00,...,0.0,0.0,22281,22280,9576,2659,NaN,NaN,NaN,Skyline Aviation SRL
7,21653,106,3051,Completed,3,NaN,Engine increased vibration,"I contact you due to the fact, that the Crew o...",2017-02-28 11:00:57+00:00,2017-02-28 09:47:57+00:00,...,0.0,0.0,CAE313329,CAE313328,14501168,5361,NaN,NaN,NaN,Air Hamburg Luftverkehrsgesellschaft mbH
8,21656,106,1598,Completed,2,NaN,EEC fault,As indicated earlier M-FISH has an ongoing iss...,2016-11-28 12:55:50+00:00,2016-11-25 12:55:50+00:00,...,0.0,0.0,11123,11124,506,2659,NaN,NaN,NaN,Business Air a.s.
9,21659,234,3106,Completed,3,NaN,EHM - Trend Notification - ESN 25028 - EMCD ca...,A trend signature (high value of EMCD) on airc...,2016-11-28 13:13:54+00:00,2016-11-28 13:13:54+00:00,...,0.0,0.0,25209,25208,6050,3096,NaN,NaN,NaN,"FlexJet, LLC."


In [24]:
for i, row in cases_df.iterrows():
    aircraft_concept = track_concept(row.aircraft_serial_number, categories['Aircraft'])
    aircraft_model_concept = track_concept(row.rolls_royce_aircraft_type, categories['Aircraft'])
    airline_concept = track_concept(row.operator, categories['Airline'])
    left_enginge_concept = track_concept(row.left_engine_serial_number, categories['Engine'])
    right_enginge_concept = track_concept(row.right_engine_serial_number, categories['Engine'])
    
    link_concepts(aircraft_concept, aircraft_model_concept)
    link_concepts(aircraft_model_concept, 'aircraft')
    link_concepts(aircraft_concept, airline_concept)
    link_concepts(airline_concept, 'aircraft')
    link_concepts(left_enginge_concept, aircraft_concept)
    link_concepts(right_enginge_concept, aircraft_concept)

## Domain Vocabulary

In [25]:
def track_term_list(line, parent, category):
    terms = line.split(',')
    terms = [t.strip() for t in terms]
    concept = track_concept(terms[0], category)
    link_concepts(concept, parent)
    
    for t in terms[1:]:
        track_term(t, concept)
    
    return concept
    

def traverse(nodes, parent, category):    
    for node in nodes:
        if type(node) is list:
            traverse(node, parent, category)
        
        if type(node) is str:
            track_term_list(node, parent, category)
        
        if type(node) is dict:
            assert len(node.keys()) == 1
            term = list(node.keys())[0]
            concept = track_term_list(term, parent, category)
            traverse(list(node.values()), concept, category)

In [26]:
with open('../data/graph/aviation/lexicon.yaml') as f:
    for category in yaml.load(f):
        traverse([category], 'root', categories[list(category.keys())[0]])

/home/phhage/anaconda3/envs/torchenv/lib/python3.7/site-packages/ipykernel_launcher.py:2: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  


# Save Output

In [28]:
import json

path = '../data/graph/aviation/preprocessing.json'

concept2neighbours = {}

for concept, neighbours in concept2neighbour.items():
    concept2neighbours[concept] = list(neighbours)

with open(path, 'w') as f:
    json_dump = json.dumps({
        'term2concept': term2concept,
        'concept2name': concept2name,
        'concept2category': concept2category,
        'concept2neighbours': concept2neighbours,
    })

    f.write(json_dump)